In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
# === CONFIGURACIÓN DE CREDENCIALES ===
usuario = "AR_POWERBI"
contrasena = "Sim2024plit!"
host = "usasuxdb263p.cotyww.com"             # Ej: "localhost" o "192.168.1.100"
puerto = "1557"              # Puerto típico de Oracle
servicio = "INEVPRO1"     # Ej: "XEPDB1", "ORCL", etc.
# === CREAR EL ENGINE DE SQLAlchemy ===
dsn = f"{host}:{puerto}/{servicio}"
engine = create_engine(f"oracle+oracledb://{usuario}:{contrasena}@{dsn}")

# === TU CONSULTA ===
query = """WITH cte AS (
SELECT s.CLIENTE, s.VENDEDOR, s.FECHA,
ROW_NUMBER() OVER (PARTITION BY s.CLIENTE ORDER BY s.FECHA DESC) AS rn
FROM VW_NET_REVENUE_TS_CU s
)
SELECT s.FECHA, cl.DESC_GRUPO_ECONOMICO, v.VENDEDOR_DESCRIPTION
FROM cte s
INNER JOIN VW_VENDEDOR v ON s.VENDEDOR = v.VENDEDOR
INNER JOIN VW_CLIENTE cl ON s.CLIENTE = cl.CLIENTE
WHERE rn = 1
ORDER BY cl.DESC_GRUPO_ECONOMICO ASC"""  # Adaptá según lo que necesites

# === ABRIR UN DBAPI2 CONNECTION Y CARGAR EN DATAFRAME ===
conn = engine.raw_connection()
try:
    vendedores = pd.read_sql(query, con=conn)
    print("✅ Carga Exitosa")
finally:
    conn.close()

C:\Users\ArnaldoCarrasco\AppData\Local\Temp\ipykernel_23828\571502614.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vendedores = pd.read_sql(query, con=conn)


✅ Carga Exitosa


In [3]:
# Supongamos que tu dataframe se llama simplit_productos
vendedores.rename(columns={
    'VENDEDOR_DESCRIPTION': 'Vendedor',
    'DESC_GRUPO_ECONOMICO': 'Grupo Economico',
    'FECHA': 'Fecha'
}, inplace=True)

vendedores

,Fecha,Grupo Economico,Vendedor
0,2025-05-17,"FARMACIA KM 47,5 S. C. S",JUAN GONZALEZ ABELLA
1,2025-05-20,FARMACIA MARK SCS,JUAN GONZALEZ ABELLA
2,2025-05-17,FARMATODO ARGENTINA S.A,JUAN GONZALEZ ABELLA
3,2025-06-10,ACOFAR COOP.FARMACEUTICA LDA.,"GATICA, MARCOS"
4,2025-05-21,"ACOSTA, CAMILA",GAYF
...,...,...,...
336,2023-04-12,"ZABALA, JUAN CARLOS",MATIAS CASTRO
337,2022-11-04,"ZAMARGUILEA, MARIA CANDELA",GAYF
338,2023-06-21,ZAPAIA CESAR BELTRAN,"ZAMARGUILEA, MARÍA CANDELA"
339,2025-05-21,"ZEIDEL, SOFIA LARA",GAYF
